In [1]:
from collections import deque
import gym
from keras import models
from myutils.performanceAnalyzation.Analytics import Analytics
import datetime
from myutils.trainingClasses.MountainCarConvolutionalTraining import MountainCarConvolutionalTraining as conv_trainor
import numpy as np
import myutils.constants.Constants as cts


analytics = Analytics()
env = gym.make('MountainCar-v0').env

#play 100 times
#load the network
time_steps_in_episode = 300
episodes = 20
model=models.load_model(cts.Constants.PATH_TO_MODELS_OFFLINE_LEARNING+'DQN_CNN_model_interation_520000.h5')

stack_depth = 4
frame_skip = 4

frames_memory = deque(maxlen=stack_depth)

for i in range(episodes):

    env.reset()

    reward_sum=0

    current_image =env.render(mode='rgb_array')
    current_frame =conv_trainor.process_image(conv_trainor ,current_image)  # the frame is an greyscale image of the current position
    current_frame = current_frame.reshape(1, current_frame.shape[0], current_frame.shape[1])
    current_state = np.repeat(current_frame,2, axis=0)
    frames_memory.extend(current_state)

    for t in range(time_steps_in_episode):

        if (t% frame_skip) == 0:
            current_state = current_state.reshape(1, current_state.shape[0], current_state.shape[1], current_state.shape[2])
            best_action = np.argmax(model.predict(current_state)[0])


        new_state_numerical, reward, done, _ = env.step(best_action)

        new_image = env.render(mode='rgb_array')
        next_frame = conv_trainor.process_image(conv_trainor,new_image)
        next_frame = next_frame.reshape(next_frame.shape[0], next_frame.shape[1])

        # current_state is a FIFO buffer so just by appending the size  of current_state is constant
        frames_memory.append(next_frame)

        new_state = np.asarray(frames_memory)

        # make the training possible only when the minimum experience was gathered

        reward_sum += reward
        current_state = new_state

        if done:
            break

    if t >= time_steps_in_episode - 1:
        print("Failed to finish task in episode {} with reward {} ".format(i, reward_sum))
    else:
        print("Success in epsoide {}, used {} time steps and obtain reward_sum : {}!".format(i, t,reward_sum))


Using TensorFlow backend.


Failed to finish task in episode 0 with reward -200.0 
Failed to finish task in episode 1 with reward -200.0 
Failed to finish task in episode 2 with reward -200.0 
Failed to finish task in episode 3 with reward -200.0 
Failed to finish task in episode 4 with reward -200.0 
Failed to finish task in episode 5 with reward -200.0 
Failed to finish task in episode 6 with reward -200.0 
Failed to finish task in episode 7 with reward -200.0 
Failed to finish task in episode 8 with reward -200.0 
Failed to finish task in episode 9 with reward -200.0 
Failed to finish task in episode 10 with reward -200.0 
Failed to finish task in episode 11 with reward -200.0 
Failed to finish task in episode 12 with reward -200.0 
Failed to finish task in episode 13 with reward -200.0 
Failed to finish task in episode 14 with reward -200.0 
Failed to finish task in episode 15 with reward -200.0 
Failed to finish task in episode 16 with reward -200.0 
Failed to finish task in episode 17 with reward -200.0 
Fa

In [ ]:
import gym
import numpy as np
from keras import models
from myutils.performanceAnalyzation.Analytics import Analytics
import datetime
import myutils.constants.Constants as cts

analytics = Analytics()
env = gym.make('MountainCar-v0')

#play 100 times
#load the network

model=models.load_model(cts.Constants.PATH_TO_MODELS_VALID_SOLUTION+ 'bestTrainNetworkIMountainCar.h5')


for i_episode in range(100):
    currentState = env.reset().reshape(1, 2)

    print("============================================")

    rewardSum=0
    time_start_episode = datetime.datetime.now()
    for t in range(200):
        action = np.argmax(model.predict(currentState)[0])

        new_state, reward, done, info = env.step(action)

        new_state = new_state.reshape(1, 2)

        currentState=new_state

        rewardSum+=reward
        if done:
            print("Episode finished after {} timesteps reward is {}".format(t+1,rewardSum))
            break

    if t >= 199:
            print("Failed to finish task in epsoide {}".format(i_episode))
    else:
        print("Success in epsoide {}, used {} iterations!".format(i_episode, t))

    time_end_episode = datetime.datetime.now()
    duration_episode = time_end_episode - time_start_episode

    analytics.add_info(rewardSum, i_episode, t, duration_episode)
print("Final testing")

Using TensorFlow backend.
